### Homework 5 - BIOINF 575

Goals:
* Reading from a file
* Creating and using basic data structures - list(), dict()
* Creating and using numpy arrays

The data file GSE53697_RNAseq_AD.txt is a tab-separated file that contains raw read counts and gene expression values in the form of RPKM values from RNA sequencing of 8 control and 9 Alzheimer's disease (AD) samples (human).

The first line in the file is the header that describes the table of values:

GeneID - Gene ID

GeneSymbol - Gene Symbol

8 columns: C1-8_raw - raw read count values for the 8 control samples

9 columns: A9-17_raw - raw read count values for the 9 Alzheimer's disease samples

8 columns: C1-8_rpkm - RPKM values for the 8 control samples

9 columns: A9-17_rpkm - RPKM values for the 9 Alzheimer's disease (AD) samples


Reads Per Kilobase of transcript per Million mapped reads (RPKM) is a normalized unit of transcript/gene expression.
This is used to normalize for sequencing depth and gene length. Different genes might have more reads just due to the fact that they have more bases that the reads can align to and that needs to be accounted for. Also, if one sample (s1) has more reads than another sample (s2) then the overall expression of all genes in s1 would be higher than in s2. Hence, the two samples cannot be compared if a normalization for sequencing depth is not performed.

Given this file, select differentially expressed genes between Alzheimer's disease and control.
There are many ways to compute differential expression.
Here, to go from expression to differential expression, for each gene you have to compute the ratio between two averages: (1) the average expression of the gene in disease and (2) the average expression of the gene in control.

Since these averages might be 0, before you compute the ratio add 1 to the numerator and the denominator.
Using the log2 on this ratio will allow us to easily see if a gene is under-expressed (negative log value) or over-expressed (positive log values) and also the base 2 of the logarithm allows us to see when the value in AD is two times higher than the one in control by using the threshold 1 for the log2ratio.

Select the differentially expressed genes by writing the following functions.

1. Create a function data_list that takes as a parameter the file name and creates three lists:
    * lst_genes - the list of genes symbols - the genes symbols are on the second column of each line in the file;
    * lst_ge_C - each element of this list is a list - the list of control RPKM values for a gene, from each line in the file, split the line by tab and take elements 19 to 27 from the list resulted from split (these are 0-based indices); 
    * lst_ge_AD - each element of this list is a list - the list of AD RPKM values for a gene, from each line in the file, split the line by tab and take elements starting from 27 from the list resulted from split (this is a 0-based index). 


Note: The first line in the file is the header line, don't use it in computation.

2. Create a function compute_row_average that takes as input a list of lists with numbers stored as text (lst_ge) transforms the lists of lists in a 2-dimensional array and returns a 1-dimensional array containing the mean of each row. 

Note: Use the function astype() for an array that contains numbers as strings to set the type to float

3. Fill in the missing code in the gene_dictionary function that computes the log 2 of the ratio between the AD and control mean values returned by the function compute_row_average, then selects the genes that have a change greater than or equal to 2 fold (absolute value >= 1 for the log2ratio), and finally computes and returns a dictionary with the selected genes and their log2ratio (each selected gene is a key in the dictionary and the corresponding log2ratio is the value). 


In [30]:
import numpy as np
import pandas as pd
import math

In [31]:
def data_list(file_name):
    """ 
    This function initializes three lists of lists.
 

    This function initializes three lists of lists, lst_genes, lst_ge_C, and lst_ge_AD. It takes a file path as a string 
    in order to locate the data for use. Each List is initalized from another function 
            lst_genes: from lstGenes
            lst_ge_C:  from lstGeC
            lst_ge_AD: from lstGeAD



    Parameters: 
    filename (string): The argument of this function is a an RNAseq data (.txt) file location as a string.
    

    Returns: 
    lst_genes (list): This output is a list of the gene symbols for every gene present in the data
                      (In order by first occurence)
    
    lst_ge_C (list): This output is a list of all of the control RPKM values for each gene, separated into lists
                     themselves
    
    lst_ge_C (list): This output is a list of all of the disease RPKM values for each gene, separated into lists
                     themselves

    """
    lst_genes=lstGenes(file_name)
    lst_ge_C=lstGeC(file_name)
    lst_ge_AD=lstGeAD(file_name)
    
    
    return lst_genes, lst_ge_C, lst_ge_AD 

In [32]:
def lstGenes(file_name):
    """ 
    This function creates a list of the gene symbols correlating to the genes present in the data file in order
    of first occurrence.
    

    This function takes a file location as a string to open and read in the data from said file. The file is then
    parsed for the gene symbols of every gene present in the data file and added to the list, symbollist.



    Parameters: 
    filename (string): The argument of this function is a an RNAseq data (.txt) file location as a string.


    Returns: 
    lst_genes (list): This output is a list of the gene symbols for every gene present in the data
    

    """
    symbolList=[]
    with open(file_name, 'r') as file:
        for line in file:
            if not line.startswith("GeneID"):
                line=line.strip()
                line=line.split('\t')
                symbolList.append(line[1])
    return(symbolList)        

In [33]:
def lstGeC(file_name):
    """ 
    This function creates a list of the control RPKM values for each gene present in the data file. Gene data is 
    separated by individual lists within the list.


    This function takes a file location as a string to locate, open, and read in the data from said file. The file is
    then parsed for the control RPKM values for each gene. All values per gene are enclosed in a list, before being added
    to the final list, C_RPKM_list, and returned.



    Parameters: 
    filename (string): The argument of this function is a an RNAseq data (.txt) file location as a string.


    Returns: 
    lst_ge_C (list): This output is a list of all of the control RPKM values for each gene, separated into lists
                     themselves    

    """
    C_RPKM_list=[]
    with open(file_name, 'r') as file:
        for line in file:
            RPKM_list=[]
            if not line.startswith("GeneID"):
                line=line.strip()
                line=line.split('\t')
                line=np.array(line)
                line=line[[19,20,21,22,23,24,25,26]]
                line = line[np.newaxis,:]
                RPKM_list.append(line)
                C_RPKM_list.append(RPKM_list)
    return(C_RPKM_list)

In [34]:
def lstGeAD(file_name):
    """ 
    This function creates a list of the disease RPKM values for each gene present in the data file. Gene data is 
    separated by individual lists within the list.


    This function takes a file location as a string to locate, open, and read in the data from said file. The file is
    then parsed for the disease RPKM values for each gene. All values per gene are enclosed in a list, before being added
    to the final list, AD_RPKM_list, and returned.



    Parameters: 
    filename (string): The argument of this function is a an RNAseq data (.txt) file location as a string.


    Returns: 
    lst_ge_C (list): This output is a list of all of the disease RPKM values for each gene, separated into lists
                     themselves    

    """
    AD_RPKM_list=[]
    with open(file_name, 'r') as file:
        for line in file:
            RPKM_list=[]
            if not line.startswith("GeneID"):
                line=line.strip()
                line=line.split('\t')
                line=np.array(line)
                line=line[[27,28,29,30,31,32,33,34,35]]
                line = line[np.newaxis,:]
                RPKM_list.append(line)
                AD_RPKM_list.append(RPKM_list)
    return(AD_RPKM_list)

In [35]:
def compute_row_average(RPMKlist):
    """ 
    This function takes a list of lists consisting of RPKM data as an argument. It outputs the mean of the data
    for each gene


    This function takes a list of lists consisting of RPKM data as an argument. The list is parsed for the numpy arrays
    containing the RPKM data for each gene and concatenates them into a 2-dimensional array by order of occurence(Each 
    row being the RPKM data for a different gene). The mean of each row is taken (therefore, the mean of the RPKM values
    for each gene is taken) and subsequently added to the array, geneRPMKmean, which is then returned.



    Parameters: 
    RPMKlist (list): The argument of this function is a list containing lists of RPKM values for the genes in the
                     data file


    Returns: 
    geneRPMKmean (array): This output is an array of the mean RPMK values for the genes in the argument list(In the
                          order of first occurence)

    """
    rows=len(RPMKlist)
    columns=len(RPMKlist[0][0])
    matrix=RPMKlist[0][0]
    for element in RPMKlist[1:]:
        matrix=np.concatenate((matrix,element[0]), axis=0)
    matrix=matrix.astype(np.float64)
    geneRPMKmean=matrix.mean(axis=1)
    return(geneRPMKmean)

In [36]:
def gene_dictionary(file_name):
    """ 
    This function creates a dictionary of genes with a 2-fold or more change in gene expession in a disease state
    compared to the control. Negative values indicate genes in which expression is two-fold or greater in the 
    control compared to the disease state. Positive values indicate genes in which expression is two-fold or
    greater in the disease state compared to the control.
    

    This function takes a file location as a string as an argument. The argument is passed on to the data_list
    function to obtain lst_genes, lst_ge_C, and lst_ge_AD. Using the compute_row_average function, the mean
    RPMK values are obtained. The log base 2 of the ratio of disease RPMK to control RPMK is determined. Genes
    with a an absolute Log base 2 ratio greater than or equal to 1 are added to the dictionary, gene_dict, along
    with their values. Negative values indicate genes in which expression is two-fold or greater in the control
    compared to the disease state. Positive values indicate genes in which expression is two-fold or greater in
    the disease state compared to the control.
    
    
    
    Parameters: 
    filename (string): The argument of this function is a an RNAseq data (.txt) file location as a string.


    Returns: 
    gene_dict (dictionary): This output of this function is a dictionary of genes with a 2-fold or more increase
                            of gene expression in disease states compared to control.

    """
    lst_genes, lst_ge_C, lst_ge_AD = data_list(file_name)
    ge_mean_C = compute_row_average(lst_ge_C)
    ge_mean_AD = compute_row_average(lst_ge_AD)
    ge_mean_C += 1
    ge_mean_AD +=1
    RPMKratio= np.divide(ge_mean_AD,ge_mean_C)
    RPMKratio_log=[]
    gene_dict=dict()
    for element in RPMKratio:
        b2log=math.log(element,2)
        #b2log=abs(b2log)
        RPMKratio_log.append(b2log)
    for element in RPMKratio_log:
        if abs(element) >= 1:
            position = RPMKratio_log.index(element)
            gene_dict[lst_genes[position]]= element     
    return gene_dict

----

In [37]:
# Do not change this cell
gene_dict = gene_dictionary("GSE53697_RNAseq_AD.txt")
gene_dict

{'B2M': -1.0831605463018765,
 'CD44': 1.1560769875383454,
 'CHI3L1': -1.8047781530898872,
 'CHI3L2': -2.707243055240693,
 'DSP': -1.1441831930663147,
 'IFI6': -1.5164312668299031,
 'GBP1': -1.5800630930378448,
 'GBP2': -1.1175208782755517,
 'GBP3': -1.0950696178929018,
 'IFIT3': -1.4148330181938937,
 'IGF2': -1.0472490118066526,
 'INDO': -1.6683566883503855,
 'CXCL10': -1.885573593462525,
 'ITIH4': -1.1331664356985078,
 'MGP': -1.6161683252017167,
 'CXCL9': -3.856847738539151,
 'OAS1': -1.6334446059020005,
 'OAS3': -1.0420785413245308,
 'RARRES3': -1.2606496379935344,
 'S100A8': -1.624071217798039,
 'S100A9': -1.6345458580287295,
 'S100A12': -1.37429296546868,
 'SAA1': -1.2798556154280047,
 'CCL2': 1.304777297411683,
 'SLPI': -2.0716382167069907,
 'TAP1': -1.0301873977834999,
 'TTR': -1.05871241958657,
 'UTY': -1.1942877312192577,
 'JARID1D': -2.0197058572193796,
 'USP9Y': -1.0501526497324956,
 'DDX3Y': -1.393925707894159,
 'ISG15': -2.6079770382911507,
 'IFI44L': -1.0851365892348082,
